# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: Unveiling the Power of DeepLearning.AI

Are you ready to dive into the world of artificial intelligence and machine learning? Look no further than DeepLearning.AI. Created by renowned expert Andrew Ng, this platform offers top-notch courses that will equip you with the knowledge and skills needed to thrive in the rapidly evolving tech industry. From deep learning to neural networks, DeepLearning.AI covers it all in an engaging and accessible way. Whether you're a beginner or an experienced professional, there's something for everyone. Join the millions of learners worldwide who have already transformed their careers with DeepLearning.AI. It's time to unlock your potential!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Are you ready to dive into the world of artificial intelligence and machine learning? Look no further than DeepLearning.AI. Created by renowned expert Andrew Ng, this platform offers top-notch courses that will equip you with the knowledge and skills needed to thrive in the rapidly evolving tech industry. From deep learning to neural networks, DeepLearning.AI covers it all in an engaging and accessible way. Whether you're a beginner or an experienced professional, there's something for everyone. Join the millions of learners worldwide who have already transformed their careers with DeepLearning.AI. It's time to unlock your potential!

--------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: Unveiling the Power of DeepLearning.AI

Are you ready to dive into the world of artificial intelligence and machine learning? Look no further than DeepLearning.AI. Created by renowned expert Andrew Ng, this platform offers top-notch courses that will equip you with the knowledge and skills needed to thrive in the rapidly evolving tech industry. From deep learning to neural networks, DeepLearning.AI covers it all in an engaging and accessible way. Whether you're a beginner or an experienced professional, there's something for everyone. Join the millions of learners worldwide who have already transformed their careers with DeepLearning.AI. It's time to unlock your potential!

--------------------------------------------------

Critic (to Meta Reviewer):

Aggregrate feedback from all reviewers and give final suggestions on the writing.
Context: 
{'Reviewer': 'SEO Analyst', 'Review': '1. Title optimization: Consider using specific keywords related to deep learning, AI, and Andrew Ng to improve search engine visibility. For example, "Master AI with Andrew Ng's DeepLearning.AI: Courses for All Skill Levels." 2. Incorporate relevant keywords: Integrate key terms like artificial intelligence, machine learning, deep learning, Andrew Ng, online courses, etc., strategically throughout the content to enhance search engine optimization. 3. Call-to-action enhancement: Strengthen the call-to-action by including actionable phrases like "Start learning today!" or "Discover your AI potential with DeepLearning.AI" to prompt user engagement and potentially improve conversion rates.'}
{'Reviewer': 'Legal Reviewer', 'Review': '1. Ensure that all claims made regarding the content of DeepLearning.AI are accurate and supported by 

## Get the summary

In [16]:
print(res.summary)

Title: Mastering AI with DeepLearning.AI and Andrew Ng

Step into the AI realm with DeepLearning.AI! Discover expert-led courses by Andrew Ng, designed to propel your career in machine learning and deep learning. From neural networks to AI applications, embark on a learning journey tailored for all skill levels. Join millions globally who have upskilled with DeepLearning.AI, a trusted platform for transformative education. Enhance your expertise, stay current with industry trends, and unleash your potential in the world of AI. Dive in today and redefine your future!


